In [1]:
import pickle
import pandas as pd
import numpy as np
import tarfile
import os
import pickle
import pandas as pd
import datetime
from tqdm import tqdm_notebook
import re
from shutil import copyfile


#топ банков по активам (не нашел россельхоз банк)
bank_list = ['sberbank', 'vtb', 'gazprombank', 'vtb24', 'alfabank', 'fk_otkritie', 'mkb']
bank_info_folder = 'TopicModelingFintech'

In [2]:
def unpack_banks(folder_with_archives, bank_list):
    for bank in bank_list:
        path_to_zip_file = folder_with_archives + '/' + bank + '.tar.gz'
        if os.path.isfile(path_to_zip_file):
            if not os.path.isdir(folder_with_archives + '/' + bank):
                tar = tarfile.open(path_to_zip_file, 'r')
                tar.extractall(folder_with_archives)
                tar.close()
        else:
            print(f'{bank} not found.')


In [3]:
unpack_banks(bank_info_folder, bank_list)

## Разметка тональности предложений

Разметим с помощью пакета deeppavlov разметим каждое предложение из отзывов

Загрузка Русентилекс

In [4]:
read_lines = []
with open('rusentilex.txt', mode='r') as f:
    for line_num, line in enumerate(f):
        if line_num < 18:
            continue
        read_lines.append([word.strip() for word in line.split(',')[:5]])
        

In [5]:
rusentilex = pd.DataFrame(read_lines, columns=['word', 'part_of_speech', 'normalized', 'sentiment', 'source'])
rusentilex.head()

,word,part_of_speech,normalized,sentiment,source
0,аборт,Noun,аборт,negative,fact
1,абортивный,Adj,абортивный,negative,fact
2,абракадабра,Noun,абракадабра,negative,opinion
3,абсурд,Noun,абсурд,negative,opinion
4,абсурдность,Noun,абсурдность,negative,opinion


In [6]:
os.chdir(bank_info_folder)

In [7]:
def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

sentiment = []
for name in bank_list:
    if os.path.isdir(name):
        print(name)
        print(datetime.datetime.now())
        cur_path = os.path.join(os.getcwd(), name, 'sentences_replies.pkl')
        sentences_replies = pd.read_pickle(cur_path)
        column = 'vocab_only'
        sentiment = pd.DataFrame(sentences_replies[column][:12_000])

        for tonality in ['negative', 'positive']:
            check_list = np.unique(rusentilex[rusentilex.sentiment == tonality].word)
            temp_list = np.zeros(len(sentiment[column]))
            for word in tqdm_notebook(check_list):
                temp_func = findWholeWord(word)
                for pos, text in enumerate(sentiment[column]):
                    if temp_func(text):
                        temp_list[pos] += 1
                
            sentiment[tonality] = temp_list
        
        sentiment.drop(column, axis=1, inplace=True)
        sentiment.to_pickle(os.path.join(os.path.join(os.getcwd(), name, name + '_sentiment.pkl')))
# sentiment

sberbank
2019-04-21 19:17:36.947656


KeyboardInterrupt: 

In [37]:

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))


def make_sentiment_zip(dest_folder):
    if not os.path.isdir(os.path.join(os.getcwd(), dest_folder)):
        os.makedirs(os.path.join(os.getcwd(), dest_folder))            
    for name in os.listdir():
        file_name = os.path.join(os.getcwd(), name, name + '_sentiment.pkl')
        if os.path.isfile(file_name):
            copyfile(file_name, os.path.join(os.getcwd(), dest_folder, name + '_sentiment.pkl'))
  
    make_tarfile(dest_folder + '.tar.gz', dest_folder)


In [39]:
make_sentiment_zip('sentilex_banki')